In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [6]:
# This is the path.py library, included in the Anaconda distribution of Python
from path import Path as p

In [69]:
PATH = "../../../data/dogscats/"
sz=224
bs = 64
n = 1000 # Number of dogs and cats to train model on
n_cats = n
n_dogs = n # We'll set these variables differently in the future for testing
# imbalanced classes
CAT = 0 # Dummy class variables
DOG = 1 # Dummy class variables

In [4]:
torch.cuda.is_available()

True

In [10]:
p(PATH).dirs()

[Path('../../../data/dogscats/models'),
 Path('../../../data/dogscats/sample'),
 Path('../../../data/dogscats/test1'),
 Path('../../../data/dogscats/tmp'),
 Path('../../../data/dogscats/train'),
 Path('../../../data/dogscats/valid')]

In [11]:
print(p(PATH + "train/").dirs())
print(p(PATH + "valid/").dirs())

[Path('../../../data/dogscats/train/cats'), Path('../../../data/dogscats/train/dogs')]
[Path('../../../data/dogscats/valid/cats'), Path('../../../data/dogscats/valid/dogs')]


In [46]:
def get_names(filelist, suffix):
    return [suffix + f.name for f in filelist]

In [47]:
train_cats = get_names(p(PATH + "train/cats/").files(), "train/cats/")
train_dogs = get_names(p(PATH + "train/dogs/").files(), "train/dogs/")
valid_cats = get_names(p(PATH + "valid/cats/").files(), "valid/cats/")
valid_dogs = get_names(p(PATH + "valid/dogs/").files(), "valid/dogs/")
print(len(train_cats))
print(len(train_dogs))
print(len(valid_cats))
print(len(valid_dogs))

11500
11500
1000
1000


In [67]:
catfiles = list(np.random.choice(train_cats, n_cats, False)) + list(valid_cats)
dogfiles = list(np.random.choice(train_dogs, n_dogs, False)) + list(valid_dogs)
fnames = catfiles + dogfiles
y = np.array([CAT]*len(catfiles) + [DOG]*len(dogfiles))
classes = list(set(y))
valid_cat_idxs = range(n, n + len(valid_cats))
valid_dog_idxs = range(n + len(valid_cats) + n, len(fnames))
val_idxs = list(valid_cat_idxs) + list(valid_dog_idxs)
                                               
# fnames: file names
# y: numpy array which contains target labels ordered by filenames.
# classes: a list of all labels/classifications
# val_idxs: index of images to be used for validation.

In [68]:
arch=resnet34
data = ImageClassifierData.from_names_and_array(PATH, fnames, y, classes, \
                                                val_idxs, bs=bs, \
                                                tfms=tfms_from_model(arch, sz))
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████████████████████████████████████████████████████████████████████████████| 18/18 [00:14<00:00,  1.20it/s]


In [64]:
lr = 0.01
epochs = 5
learn.fit(lr, epochs)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.184161   0.053327   0.984     
    1      0.092293   0.047834   0.9845                                                                                
    2      0.062073   0.044495   0.9855                                                                                
    3      0.046561   0.042303   0.986                                                                                 
    4      0.038997   0.043131   0.985                                                                                 



[array([0.04313]), 0.985]

In [56]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
lrf=learn.lr_find()

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.896867   0.087619   0.9765    



In [59]:
tfms = tfms_from_model(resnet34, sz, aug_tfms=transforms_side_on, max_zoom=1.1)

In [60]:
data = ImageClassifierData.from_names_and_array(PATH, fnames, y, classes, \
                                                val_idxs, bs=bs, \
                                                tfms=tfms)
learn = ConvLearner.pretrained(arch, data, precompute=True)